# Plug-in ML Model with Gmail

In [9]:
import turicreate as tc

In [10]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [11]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly','https://www.googleapis.com/auth/gmail.modify']

## Load Saved Prediction Model

In [12]:
spam_model=tc.load_model('final_prediction_model')

In [13]:
def spam(ids,addLabel='Label_3',removeLabel='INBOX'):
    batch_modify={
        "ids":[ids],
        "addLabelIds":[addLabel],
        "removeLabelIds":[removeLabel]
    }
    return batch_modify

In [14]:
def predict(text):
    data=tc.load_sframe([text])
    data['word_count']=tc.text_analytics.count_words(data['X1'])
    p=spam_model.predict(data,output_type='probability')
    del(data)
    return p[0]

## Reading Mails from Inbox and Labeling them as Spam after Prediction

In [15]:
def main():
    
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
   
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    
    try:
        results=service.users().messages().list(userId='me',maxResults=50,labelIds=['INBOX']).execute()
        messages=results.get('messages',[])
        for message in messages:
            msg=service.users().messages().get(userId='me',id=message['id']).execute()
            text=msg['snippet']
            if predict(text)>0.5:
                new_label=spam(message['id'])
                new_batch=service.users().messages().batchModify(userId='me',body=new_label).execute()
    except Exception as e:
        print("Error occured : {e}")

In [16]:
main()
print('Detection complete')

Detection complete
